In [141]:
# Read in some tweets
import csv
jack_reader = csv.reader(open('data/realdonaldtrump.csv', 'r'))

columns = next(jack_reader)

In [142]:
jack_tweets = list(jack_reader)

In [143]:
# Filter out retweets
jack_tweets_no_rts = list(filter(lambda x: not x[1].startswith('RT'), jack_tweets))

for i in range(10):
    print(jack_tweets_no_rts[i][1])


Thank you Reno, Nevada. 
NOTHING will stop us in our quest to MAKE AMERICA SAFE AND GREAT AGAIN! #AmericaFirst… https://t.co/A4eeHoCbGS
Join me live in Reno, Nevada!
https://t.co/T4bf1hrxaA https://t.co/EPqRXHa1CM
JOIN ME TOMORROW!
MINNESOTA • 2pm
https://t.co/WcgLh4prS7

MICHIGAN • 6pm
https://t.co/9BqGVKNNrt

VIRGINIA • 9:30p… https://t.co/A1oVhCrT6t
#DrainTheSwamp!
https://t.co/z68vGp9Bvf
Top Clinton Aides Bemoan Campaign ‘All Tactics,’ No Vision: https://t.co/mHYvQtIq78
‘Must Act Immediately’: Clinton Charity Lawyer Told Execs They Were Breaking The Law
https://t.co/hsi4qhqTV1
Watch Coach Mike Ditka- a great guy and supporter tonight at 8pmE on #WattersWorld with @jessebwatters @FoxNews.
Thank you Wilmington, North Carolina. We are 3 days away from the CHANGE you've been waiting for your entire life!… https://t.co/6ZJZRBfLST
Thank you for the incredible support this morning Tampa, Florida! #ICYMI- watch here: https://t.co/q43kHf7MoE https://t.co/1GscFNaV4L


In [4]:
# Canonicalize the tweet text as lowercase
import re
jack_tweets_no_rts_lowercase = [tweet[1].lower() for tweet in jack_tweets_no_rts]

# Canonicalize links to "URL" and @mentions to "USER"
jack_tweets_no_rts_lowercase = [re.sub(r'(https?:\/\/t\.co\/\w+)', 'URL', tweet) for tweet in jack_tweets_no_rts_lowercase]
jack_tweets_normalized = [re.sub(r'(@\w+)', 'USER', tweet) for tweet in jack_tweets_no_rts_lowercase]

for i in range(10):
    print(jack_tweets_no_rts_lowercase[i])
    print("  -> {}".format(jack_tweets_normalized[i]))


  -> 
thank you reno, nevada. 
nothing will stop us in our quest to make america safe and great again! #americafirst… URL
  -> thank you reno, nevada. 
nothing will stop us in our quest to make america safe and great again! #americafirst… URL
join me live in reno, nevada!
URL URL
  -> join me live in reno, nevada!
URL URL
join me tomorrow!
minnesota • 2pm
URL

michigan • 6pm
URL

virginia • 9:30p… URL
  -> join me tomorrow!
minnesota • 2pm
URL

michigan • 6pm
URL

virginia • 9:30p… URL
#draintheswamp!
URL
  -> #draintheswamp!
URL
top clinton aides bemoan campaign ‘all tactics,’ no vision: URL
  -> top clinton aides bemoan campaign ‘all tactics,’ no vision: URL
‘must act immediately’: clinton charity lawyer told execs they were breaking the law
URL
  -> ‘must act immediately’: clinton charity lawyer told execs they were breaking the law
URL
watch coach mike ditka- a great guy and supporter tonight at 8pme on #wattersworld with @jessebwatters @foxnews.
  -> watch coach mike ditka- a gre

In [5]:
import nltk
from nltk import word_tokenize
# note: need to nltk.download() all the models the first time aroudn

# Frequency distribution of words.
main_dist = nltk.FreqDist([])
for tweet in jack_tweets_normalized:
    tokens = word_tokenize(tweet)
    
    # We don't want to count the mentions and hashtags
    # tokens = list(filter(lambda x: not (x[0] == '@' or x[0] == '#'), tokens))
    
    main_dist.update(tokens)

In [6]:
main_dist.most_common()[0:40]

[('.', 1095),
 ('!', 972),
 ('URL', 875),
 (',', 863),
 ('the', 846),
 ('#', 753),
 ('to', 517),
 ('USER', 487),
 ('and', 468),
 ('in', 442),
 ('is', 363),
 ('of', 359),
 ('you', 351),
 (':', 351),
 ('a', 345),
 ('i', 295),
 ('will', 280),
 ('hillary', 276),
 ('for', 259),
 ('thank', 234),
 ('on', 229),
 ('``', 191),
 ('clinton', 191),
 ('-', 190),
 ('we', 190),
 ('at', 173),
 ('great', 172),
 ("''", 172),
 ('be', 163),
 ('that', 157),
 ('crooked', 151),
 ('me', 145),
 ('are', 143),
 ('it', 139),
 ("'s", 138),
 (';', 133),
 ('&', 132),
 ('amp', 131),
 ('with', 126),
 ('by', 117)]

In [7]:
import hmac
import binascii
import struct
from nltk import word_tokenize

hash_key = "legomystego"

def tweet_hash(tweet):
    """
    Implement a keyed hash function according to section 3.3:
    
    1. Generate a keyed hash digest (HMAC-MD5) for each word
    2. Get the last four bits of the hash
    3. Bitwise rotate each value according to its position in the tweet
    4. XOR all the values together
    """
    tokens = word_tokenize(tweet)
    
    token_hashes = []
    tweet_hash = 0     # Start with 0, the XOR identity
    for n, token in enumerate(tokens):
        # Generate the keyed hash with the given key
        m = hmac.new(hash_key.encode(), msg=token.encode())
        
        m_hash = m.digest()
        
        # Get the last nibble of the hash
        m_bits = int(m_hash[-1]) & 0x0f
        
        
        # ROT-N for the position in the tweet
        for i in range(n):
            m_bits_shifted = m_bits << 1
            m_bits_overflow = m_bits_shifted & 0xf0
            m_bits_lower = m_bits_shifted & 0x0f
            
            m_bits = m_bits_lower + (m_bits_overflow >> 4)
            
        tweet_hash ^= m_bits
        token_hashes.append(m_bits)
        
    return hex(tweet_hash)
    

In [8]:
tweet_hash("ferociosu world")

'0xd'

In [9]:
tweet_hash("world hello")

'0x3'

In [10]:
tweet_hash("magical wonderful magic")

'0x6'

In [119]:
# Substitutions
import sqlite3
import dataset
RULES_DATABASE_URI = "sqlite:///ppdb/rules.db"

rules_database = dataset.connect(RULES_DATABASE_URI)
lexical_rules = rules_database["lexical"]


In [120]:
print(list(lexical_rules.find(source="attraction")))
rules = list(lexical_rules.all())

[OrderedDict([('id', 57148), ('source', 'attraction'), ('target', 'attractiveness'), ('features', '1.0')])]


In [121]:
def get_synonyms(rules, token):
    """
    Get a list of synonyms for the given token
    
    Args:
    
    token: a string of the token for which to find synonyms
    
    Return:
    
    a list of synonyms
    """
    return list(set([r['target'] for r in list(rules.find(source=token))]))

# Test
get_synonyms(lexical_rules, "attraction")

['attractiveness']

In [122]:

def get_synonyms_for_list(rules, tokens):
    """
    Generate synonyms for each token.
    
    Return as a dictionary
    """
    synonyms = {}
    
    for token in tokens:
        current_synonyms = get_synonyms(rules, token)
        
        if current_synonyms:
            synonyms[token] = current_synonyms
        
    return synonyms

In [123]:
print(get_synonyms_for_list(lexical_rules, "I'm fairly attractive".split()))
print(get_synonyms_for_list(lexical_rules, "What movie are you seeing this evening".split()))

{'fairly': ['equitably'], 'attractive': ['appealing']}
{'movie': ['film'], 'evening': ['night']}


In [136]:
import kenlm
model = kenlm.LanguageModel('data/tweets.klm')


def get_possible_cover_tweet(rules, tweet):
    tokens = word_tokenize(tweet)
    synonyms = get_synonyms_for_list(rules, tokens)

    # A list of tuples (score, cover tweet)
    possibilities = []

    for token, alternatives in synonyms.items():
        for alt in alternatives:
            alt_tweet = tweet.replace(token, alt)
            alt_score = model.score(alt_tweet)
            
            possibilities.append((alt_score, alt_tweet, tweet_hash(alt_tweet)))
                        
    possibilities = sorted(possibilities, reverse=True)
    
    for possibility in possibilities:
        print(possibility)
        
    return possibilities

In [137]:
possibilities = get_possible_cover_tweet(lexical_rules, "what movie do you want to see this evening")

(-21.025192260742188, 'what movie do you wanna to see this evening', '0xa')
(-23.441560745239258, 'what movie do you want to see this night', '0x4')
(-26.4293212890625, 'what film do you want to see this evening', '0xd')


In [138]:
possibilities = get_possible_cover_tweet(lexical_rules, "i am going to see grandma")


(-8.447860717773438, 'i am going to see granny', '0x8')
(-11.36048412322998, 'i ben going to see grandma', '0x1')
(-11.36048412322998, 'i &apos;m going to see grandma', '0xd')
(-12.668585777282715, 'i am going to see grandmother', '0x9')


In [139]:
o2m_rules = rules_database["o2m"]

In [140]:
possibilities_o2m = get_possible_cover_tweet(o2m_rules, "i am going to see grandma")
possibilities_lexical = get_possible_cover_tweet(o2m_rules, "i am going to see grandma")

possibilities = []
possibilities.extend(possibilities_o2m)
possibilities.extend(possibilities_lexical)
possibilities = list(set(possibilities))

(-9.051159858703613, 'i am going to refer to grandma', '0xd')
(-9.095948219299316, 'i am going to see section grandma', '0x8')
(-9.444210052490234, 'i am going to get it grandma', '0x0')
(-9.453298568725586, 'i am going to consult with grandma', '0xa')
(-10.008275985717773, 'i am going to look at grandma', '0x6')
(-10.212509155273438, 'i am going to see the section grandma', '0x4')
(-10.326525688171387, 'i am going to be seen grandma', '0xa')
(-10.354252815246582, 'i am going to colonial see grandma', '0x9')
(-10.354252815246582, 'i am going to &apos;s see grandma', '0x5')
(-10.354252815246582, 'i am going to &apos;il see grandma', '0x9')
(-10.388155937194824, 'i am going to talk to grandma', '0x1')
(-10.44310474395752, 'i am going to cf . grandma', '0x3')
(-10.533917427062988, 'i am going to to watch grandma', '0x2')
(-10.592903137207031, 'i am &apos;s gonna to see grandma', '0xd')
(-10.592903137207031, 'i am &apos;re gonna to see grandma', '0xd')
(-10.592903137207031, 'i am &apos;m g

In [135]:
possibilities = sorted(possibilities, reverse=True)

In [ ]:
# TODO: these are possibities within the  the language model